# JavaScript Microtask Queue – Deep Dive for Students

## 1. What is a Microtask?
A **microtask** is a small unit of work that executes **immediately after the current JavaScript stack finishes** but **before** the event loop continues to the next macrotask.

**Common characteristics:**
- Always run before the next macrotask.
- All queued microtasks are drained in order, and if a microtask queues more microtasks, they are added to the same drain cycle.
- Used for operations that must happen ASAP without yielding control back to the browser.

---

## 2. Common Microtask Sources in Browsers

### a) Promises
- `.then`, `.catch`, `.finally` callbacks.

```js
Promise.resolve().then(() => console.log("promise microtask"));
```

### b) `queueMicrotask`
- Explicitly schedule a microtask without creating a Promise.

```js
queueMicrotask(() => console.log("queueMicrotask microtask"));
```

### c) `MutationObserver`
- Observes changes to the DOM and schedules its callback as a microtask.

```js
const target = document.getElementById("demo");
const observer = new MutationObserver(() => console.log("mutation observer microtask"));
observer.observe(target, { childList: true });
target.textContent = "changed";
```

---

## 3. How Microtasks Fit in the Event Loop

**Rule:** After each macrotask, the browser runs **all** queued microtasks before starting the next macrotask.

Order of execution:
1. Execute synchronous code on the call stack.
2. Drain the microtask queue.
3. Render (if needed).
4. Move to the next macrotask.

---

## 4. Mini Demo Comparing Microtasks and Macrotasks
```html
<script>
console.log("script start");

setTimeout(() => console.log("timeout (macro)"), 0);

Promise.resolve().then(() => console.log("promise 1 (micro)"));
Promise.resolve().then(() => console.log("promise 2 (micro)"));

queueMicrotask(() => console.log("queueMicrotask (micro)"));

console.log("script end");
</script>
```
**Expected order:**
```
script start
script end
promise 1 (micro)
promise 2 (micro)
queueMicrotask (micro)
timeout (macro)
```

---

## 5. Key Differences vs. Macrotasks
- **Microtasks**: High priority, run before rendering.
- **Macrotasks**: Lower priority, run after rendering and user interaction.

**Tip for teaching:** Show how excessive microtasks can delay rendering by flooding the queue before the browser can paint.

```js
for (let i = 0; i < 1000; i++) {
  queueMicrotask(() => {});
}
console.log("Done scheduling microtasks");
```


---

# Microtask Queue — Deep Dive for Students

## 1. What is a Microtask?
A **microtask** is a small job that runs **right after** the current JavaScript stack finishes and **before** the event loop takes the next macrotask. After each macrotask, the browser flushes **all** queued microtasks (including any that were queued by other microtasks) before continuing.

**Key properties:**
- Runs ASAP after the current synchronous work.
- All microtasks are flushed before the next macrotask.
- New microtasks queued during a flush are executed in the same flush.

---

## 2. Common Microtask Sources in Browsers

### a) Promises
- `Promise.then`, `Promise.catch`, `Promise.finally`
```js
Promise.resolve().then(() => console.log("promise microtask"));
```

### b) `queueMicrotask(fn)`
- Explicit API to schedule a microtask without creating a Promise.
```js
queueMicrotask(() => console.log("queueMicrotask microtask"));
```

### c) `MutationObserver`
- DOM mutation notifications delivered as microtasks at the end of the current task.
```js
const mo = new MutationObserver(() => console.log("observer microtask"));
mo.observe(node, { childList: true, characterData: true, subtree: true });
node.textContent = "mutated"; // schedules observer microtask
```

---

## 3. Ordering Rules
1) Finish current synchronous code.
2) **Flush all microtasks** (FIFO). If a microtask queues more microtasks, keep flushing.
3) Only when the microtask queue is empty, pick the next **macrotask**.

**Implication:** Promises/microtasks run before `setTimeout(..., 0)` and most other tasks.

---

## 4. Why Use Microtasks?
- **ASAP continuation** after current logic (e.g., thenable chains).
- **Batching** state updates deterministically before the next render/task.
- **Deterministic order** relative to timers/events.

---

## 5. Gotchas
- **Starvation:** Flooding microtasks can delay rendering and input handling.
- **Unhandled rejections:** Rejected Promises without a handler trigger the `unhandledrejection` event.
- **Observer timing:** `MutationObserver` callbacks run after DOM mutations in the same task, often *after* earlier queued Promise microtasks.

---

## 6. Cheatsheet
- **Immediate follow-up after current call?** → Microtask (`queueMicrotask`, `Promise.then`).
- **Need to yield to UI / allow paint/input?** → Macrotask (`setTimeout(0)`, `MessageChannel`).

---

## 7. HTML Demos

### (1) Ordering: Promises beat timeouts
```html
<script>
console.log('start');
setTimeout(() => console.log('timeout (macro)'), 0);
Promise.resolve().then(() => console.log('promise-1 (micro)'));
queueMicrotask(() => console.log('queueMicrotask (micro)'));
Promise.resolve().then(() => console.log('promise-2 (micro)'));
console.log('end');
// start → end → promise-1 → queueMicrotask → promise-2 → timeout
</script>
```

### (2) Microtask flush cascade
```html
<script>
Promise.resolve().then(() => {
  console.log('A (micro)');
  queueMicrotask(() => console.log('B (micro from micro)'));
});
// A then B in the same flush, before any macrotask
</script>
```

### (3) MutationObserver as microtask
```html
<div id="t">text</div>
<script>
const t = document.getElementById('t');
const mo = new MutationObserver(() => console.log('observer (micro)'));
mo.observe(t, { childList: true, characterData: true, subtree: true });
Promise.resolve().then(() => console.log('promise (micro)'));
t.textContent = 'mutated'; // schedules observer
console.log('mutated sync');
// Expected: mutated sync → promise (micro) → observer (micro)
</script>
```

### (4) Starvation demo (don’t do this in prod)
```html
<button id="flood">Flood microtasks</button>
<script>
flood.onclick = () => {
  console.log('start flood');
  for (let i = 0; i < 5000; i++) {
    Promise.resolve().then(() => {});
  }
  setTimeout(() => console.log('timeout after flood (macro)'), 0);
  console.log('scheduled flood');
};
// UI may feel stuck until all microtasks complete; only then timeout runs
</script>
```

---

## 8. Best Practices
- Use `queueMicrotask` for intention-revealing, promise-free microtasks.
- Chunk heavy work with macrotasks to keep the UI responsive.
- Always attach `.catch` (or `try/catch` with `await`) to avoid unhandled rejections.
